# Systems Immunogenetics Project

## WNV Flow Cytometry Data Cleaning Workflow

### McWeeney Lab, Oregon Health & Science University

#### Author: Michael Mooney (mooneymi@ohsu.edu)

## Introduction

This document will walk through the steps for parsing and cleaning flow cytometry spreadsheets. The result will be a single R dataframe containing all the data, with standardized column names and formatting.

Required Files:

- This notebook (`SIG_WNV_Flow_Data_Cleaning.ipynb`): [[Download here]](https://raw.githubusercontent.com/mooneymi/systems_immunogenetics/master/SIG_WNV_Flow_Data_Cleaning.ipynb)
- The R script `flow_data_cleaning_functions.r`: [[Download here]](https://raw.githubusercontent.com/mooneymi/systems_immunogenetics/master/flow_data_cleaning_functions.r)
- The text file containing all flow variables (`final_flow.txt`): [[Download here]](https://raw.githubusercontent.com/mooneymi/systems_immunogenetics/master/data/final_flow.txt)

** Note: this notebook can also be downloaded as an R script (only the code blocks seen below will be included): [[Download R script here]](https://raw.githubusercontent.com/mooneymi/systems_immunogenetics/master/SIG_WNV_Flow_Data_Cleaning.r)

**All code is available on GitHub: [https://github.com/mooneymi/systems_immunogenetics](https://github.com/mooneymi/systems_immunogenetics)** 

If you are not familiar with Jupyter Notebooks, I've created a short tutorial to get you up and running quickly. There is also plenty of documentation online:

1. [Jupyter for R Tutorial](http://nbviewer.jupyter.org/github/mooneymi/jupyter_notebooks/blob/master/r/Getting_Started_R.ipynb)
2. [Jupyter Documentation](http://jupyter.org/)
3. [Conda and R](https://www.continuum.io/conda-for-r)

#### Up Next: Plotting Flow Cytometry Data

After finishing this workflow you will have a cleaned dataset ready for exploration and analysis. A notebook with code examples for plotting the data (including interactive plots created with the Shiny library) is available here: [[Flow Data Plotting Workflow]](http://nbviewer.jupyter.org/github/mooneymi/systems_immunogenetics/blob/master/SIG_WNV_Flow_Data_Plotting.ipynb) 

## Step 1. Prepare the Input Files

Sometimes, a bit of manual cleaning of the spreadsheets is necessary for the parsing function to work correctly.

1. Remove special characters (e.g. ¥), and set to NA (blank). It seems this character is entered when a parent population has zero cells, and therefore percentages for sub-populations are meaningless.
2. There should be no ID column in sheets other than the Treg panel (IDs will be created automatically)
3. All column names should match those in the `final_flow.txt` file (unexpected columns will be printed to the screen to alert the user). If any of the following columns are missing, errors will occur when merging the different panels: 'UNC strain', 'UW strain', 'RIX_ID' (changed from 'Mouse #'), 'Timepoint', 'Tissue' (changed from 'Organ'), 'Total Cell Count'.
4. Note that sometimes columns/rows look 'empty' in Excel, but they actually contain spaces (or even formatting) that R will recognize. If these 'empty' columns/rows are not skipped when R reads the file it can cause problems, so they may need to be manually deleted.
5. It is also a good idea to "Unfreeze Panes' and remove any splits within the spreadsheets. This can sometimes cause problems when reading the data.

## Step 2. Load the Necessary R Libraries and Functions

There are a number of functions in the accompanying R script (`flow_data_cleaning_functions.r`) necessary for parsing and then processing the flow cytometry data:

1. `read_flow_exp_file()`: Parses a flow spreadsheet and creates an R dataframe.
2. `fix_column_names()`: Standardizes the column names of the above dataframe.
3. Functions for calculating cell counts and ratios:
    - `calc_treg_counts()`
    - `calc_tcell_counts()`
    - `calc_ics_counts()`
    - `calc_ics_percent_ratios()`
    - `calc_ics_count_ratios()`
4. `clean_inf_nan()`: Sets any infinite or NaN values to NA.

More information on each of these functions is available by calling the `describe()` function. For example, the following command will print documentation for the `read_flow_exp_file()` function:

    describe(read_flow_exp_file)

Remember that, in addition to the help documentation provided with `describe()`, you can view the actual function definitions at any time by simply typing the function name without parentheses (e.g. `describe`) at the command prompt.

In [1]:
## Load functions for parsing the flow cytometry spreadsheets
## The gdata library is necessary for reading Excel spreadsheets; it will be loaded as well.
source('flow_data_cleaning_functions.r')

gdata: read.xls support for 'XLS' (Excel 97-2004) files ENABLED.

gdata: read.xls support for 'XLSX' (Excel 2007+) files ENABLED.

Attaching package: ‘gdata’

The following object is masked from ‘package:stats’:

    nobs

The following object is masked from ‘package:utils’:

    object.size



In [2]:
## View help documentation on the functions listed above
describe(read_flow_exp_file)


This function parses flow cytometry data from an Excel workbook.

Parameters:
f: The Excel file name.
cn_expected: A character vector containing the expected column names.

Returns:
A dataframe containing the processed flow cytometry data.



## Step 3. Read the Data into R

In [3]:
## Load all expected flow variables (expected column names)
flow_cn = read.delim('./data/final_flow.txt', sep='\t', as.is=T, header=F)
flow_cn = flow_cn[,1]

In [4]:
## Move to the directory holding the data
flow_dir = "~/Documents/MyDocuments/SystemsImmunogenetics/WNV/Lund_Flow_fixed_Nov_13"
setwd(flow_dir)

In [5]:
## Get a list of data files to read (in this case all flow spreadsheets begin with the prefix 'Expt')
flow_files = list.files('.', pattern="Expt.*\\.xls")

In [6]:
print(flow_files)

[1] "Expt 76 cell counts 11_13_15_fixedMM.xlsx"
[2] "Expt 77 cell counts 120815_fixedMM.xlsx"  
[3] "Expt 78 cell counts 01_11_16_fixedMM.xlsx"


In [7]:
## Iterate through all the files, parse each, and merge all data into a single dataframe
i = 1
for (file in flow_files) {
    print(file)
    flow_dat = read_flow_exp_file(file, flow_cn)
    
    ## Check if there are any unexpected columns
    new_columns = setdiff(colnames(flow_dat), flow_cn)
    if (length(new_columns) > 0) {
        flow_cn = c(flow_cn, new_columns)
    }
    if (i > 1) {
        ## Fill extra columns with NAs
        for (col in new_columns) {
            flow_all[,col] = NA
        }
        ## Merge data
        flow_all = rbind(flow_all[,flow_cn], flow_dat[,flow_cn])
    } else {
        flow_all = flow_dat
    }
    i = i + 1
}

[1] "Expt 76 cell counts 11_13_15_fixedMM.xlsx"
[1] "Treg panel"
[1] "CD8 d21"
[1] "ICS panel"
[1] "Expt 77 cell counts 120815_fixedMM.xlsx"
[1] "Treg panel"
[1] "CD8 d21"
[1] "ICS panel"
[1] "Expt 78 cell counts 01_11_16_fixedMM.xlsx"
[1] "Treg panel"
[1] "CD8 d7"
[1] "ICS panel"


In [8]:
## Check the dimensions of the dataframe
dim(flow_all)

[1]  94 277

In [9]:
## Check that all expected columns are present
setdiff(flow_cn, colnames(flow_all))

character(0)

In [10]:
setdiff(colnames(flow_all), flow_cn)

character(0)

## Step 4. Clean and Reformat the Data

In [11]:
## Order columns, add Lab column and fix formatting
flow_all = flow_all[, flow_cn]
flow_all$Lab = "Lund"

flow_all$ID = gsub(" ", "", flow_all$ID)
flow_all$ID = gsub("X", "x", flow_all$ID)
flow_all$Mating = gsub(" ", "", flow_all$Mating)
flow_all$Mating = gsub("X", "x", flow_all$Mating) 
flow_all$UW_Line = as.numeric(flow_all$UW_Line)

In [12]:
## Check for duplicate IDs
new_flow_ids = paste(flow_all$ID, flow_all$Tissue, sep='_')
sum(duplicated(new_flow_ids))

[1] 0

## Step 5. Combine with Previous Data (Optional)

In [16]:
## Read the previously cleaned data
## Note: you may have to change the file path
flow_prev = read.xls('../Cleaned_Data_Releases/9-Nov-2015/Lund_Flow_Full_3-Nov-2015_final.xlsx', 
                     as.is=T, na.strings=c(""," ", "NA", "#DIV/0!"))

In [17]:
## Check for duplicate IDs
dup_ids1 = intersect(flow_prev$ID[flow_prev$Tissue=='brain'], flow_all$ID[flow_all$Tissue=='brain'])
dup_ids2 = intersect(flow_prev$ID[flow_prev$Tissue=='spleen'], flow_all$ID[flow_all$Tissue=='spleen'])

## Overwrite old data with new
idx1 = which(flow_prev$ID %in% dup_ids1 & flow_prev$Tissue=='brain')
idx2 = which(flow_prev$ID %in% dup_ids2 & flow_prev$Tissue=='spleen')
idx_dups = c(idx1, idx2)
print(length(idx_dups))
idx_to_keep = setdiff(1:nrow(flow_prev), idx_dups)
flow_prev = flow_prev[idx_to_keep,]
flow_all = rbind(flow_prev[, flow_cn], flow_all[, flow_cn])

[1] 0


In [18]:
## Check the dimensions of the dataframe
dim(flow_all)

[1] 1655  277

## Step 6. Calculate Cell Counts and Ratios

In [20]:
## Change all data columns to numeric
for (i in 11:277) {
    flow_all[,i] = as.numeric(flow_all[,i])
}

In [21]:
## Calculate cell counts and ratios
flow_full = flow_all
flow_full = calc_treg_counts(flow_full)
flow_full = calc_tcell_counts(flow_full)
flow_full = calc_ics_counts(flow_full)
flow_full = calc_ics_percent_ratios(flow_full)
flow_full = calc_ics_count_ratios(flow_full)
flow_full = clean_inf_nan(flow_full)

## Step 7. Save the Data

In [ ]:
## Save R data file
write.table(flow_full, file='Lund_Flow_Full_11-Jan-2016_final.txt', col.names=T, row.names=F, quote=F, sep='\t', na='')
save(flow_full, file='lund_flow_full_11-jan-2016_final.rda')

### Up Next: Plotting Flow Cytometry Data

Code for plotting this data is available here: [[Flow Data Plotting Workflow]](http://nbviewer.jupyter.org/github/mooneymi/systems_immunogenetics/blob/master/SIG_WNV_Flow_Data_Plotting.ipynb)

#### Last Updated: 28-Jan-2016